In [18]:
import torch 
import numpy as np
from botorch.models import SingleTaskGP
from botorch  import fit_gpytorch_model

from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.monte_carlo import qExpectedImprovement

from botorch.optim import optimize_acqf

from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm
from scipy.optimize import minimize
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF


In [19]:
f1_inputs = np.load("initial_inputs1.npy")
f1_outputs = np.load("initial_outputs1.npy")
f1_inputs_2 = np.load("initial_inputs2.npy")
f1_outputs_2 = np.load("initial_outputs2.npy")

In [20]:
f1_inputs = np.concatenate((f1_inputs, f1_inputs_2)) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.201607 , 0.5658698]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.785688, 0.49287]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.701023, 0.702999]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.767155, 0.801202]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.756262, 0.621414]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.721023, 0.682999]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.707689, 0.722999]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.716578, 0.705221]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.703245, 0.682999]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.609401, 0.605101]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.629401, 0.589545]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.629401, 0.616212]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.649401, 0.622878]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.631623, 0.636212]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.611623, 0.656212]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.642734, 0.616212]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.651623, 0.620656]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.642734, 0.616212]]))) 

f1_outputs = np.concatenate((f1_outputs,f1_outputs_2)) 
f1_outputs = np.append(f1_outputs,-5.9027047474104416e-49)
f1_outputs = np.append(f1_outputs,1.4694233057027134e-30)
f1_outputs = np.append(f1_outputs,6.013687404741693e-08)
f1_outputs = np.append(f1_outputs,1.029025518149088e-34)
f1_outputs = np.append(f1_outputs,3.1568882670317357e-12)
f1_outputs = np.append(f1_outputs,9.594441158769377e-09)
f1_outputs = np.append(f1_outputs,4.4984959147048566e-11)
f1_outputs = np.append(f1_outputs,6.25317474883913e-10)
f1_outputs = np.append(f1_outputs,1.2990004780975896e-06)
f1_outputs = np.append(f1_outputs,0.1971687803797823)
f1_outputs = np.append(f1_outputs,0.0019282695144167755)
f1_outputs = np.append(f1_outputs,1.378994567184674)
f1_outputs = np.append(f1_outputs,0.6197596252221214)
f1_outputs = np.append(f1_outputs,1.6960217184989461)
f1_outputs = np.append(f1_outputs,0.12963516015284113)
f1_outputs = np.append(f1_outputs,0.8437573006254239)
f1_outputs = np.append(f1_outputs,0.4426566990166618)
f1_outputs = np.append(f1_outputs,0.8437573006254239)


In [21]:
X_max= f1_inputs[np.argmax(abs(f1_outputs))]
print(X_max)

[0.631623 0.636212]


In [22]:
Y_max= f1_outputs[np.argmax(abs(f1_outputs))]
print(Y_max)

1.6960217184989461


In [23]:
variable_value = np.linspace(1, 10, 5)
n_dims = 2 
beta =0.01 # 1.6262626262626263
np.random.seed(42)
train_X = torch.tensor(f1_inputs)
train_Y = torch.tensor(f1_outputs).reshape(len(f1_outputs),-1)
best_y = np.max(train_Y.numpy())
best_x = train_X[np.argmax(train_Y.numpy())]
noise_level =0.01# 1.0
candidate_list = []
best_x_list = []

#train_X = train_X1[0].reshape(8,-1)
#train_Y = train_Y1[0].reshape(1,-1)

for i in range(len(variable_value)):
    #noise_level = variable_value[i]
   

    def function_1(x1,x2, noise_level=noise_level):
        result = np.sin(5 * x1+4*x2) * (1 - np.tanh(x1 ** 2) - np.tanh(x2))\
        + np.random.randn() * noise_level
        return result
    
    
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    #UCB = UpperConfidenceBound(gp, beta=beta)
    EI = qExpectedImprovement(model=gp , best_f=best_y)

    bounds = torch.stack([torch.zeros(n_dims), torch.ones(n_dims)])
    candidate, _  = optimize_acqf(
        EI, bounds=bounds, q=1, num_restarts=50, raw_samples=500,
    )


    x1,x2 = candidate.numpy().ravel().tolist()
    # Get the next value for the function from the script

    y_next = function_1(x1,x2)

    if y_next > best_y:
        best_y = y_next
        best_x = x1,x2


    y_next = torch.tensor(y_next).unsqueeze(-1)

    train_X = torch.cat([train_X,candidate])
    train_Y = torch.cat([train_Y,y_next.unsqueeze(-1)])
    candidate_list.append(candidate) 
    best_x_list.append(best_x)
    #print(f"Next point: {candidate} \n Next Y {y_next}  Best_y, {best_y}")
    #print(f"beta :{beta} noise_level :{noise_level}")
    #print(f"Next point: {candidate} \n Next Y {y_next}")
    #print(f"best_x:{best_x}Best_y,{best_y}")
    

In [16]:
candidate_list

[tensor([[0.6357, 0.1686]]),
 tensor([[0.8352, 0.2574]]),
 tensor([[0.8784, 0.5120]]),
 tensor([[0.7342, 0.9178]]),
 tensor([[0.8999, 0.5900]])]

In [17]:
best_x_list

[tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64)]

In [39]:
candidate_list

[tensor([[0.8565, 0.4396]]),
 tensor([[0.2769, 0.5835]]),
 tensor([[0.5761, 0.5033]])]

In [40]:
best_x_list

[(0.8564834594726562, 0.43957212567329407),
 (0.8564834594726562, 0.43957212567329407),
 (0.8564834594726562, 0.43957212567329407)]

In [42]:
best_x_list

[(0.49093690514564514, 0.0673556923866272),
 (0.49093690514564514, 0.0673556923866272),
 (0.5164217352867126, 0.031376034021377563)]

In [46]:
best_x_list

[tensor([0.5965, 0.6141], dtype=torch.float64),
 tensor([0.5965, 0.6141], dtype=torch.float64),
 tensor([0.5965, 0.6141], dtype=torch.float64)]

In [57]:
best_x_list

[tensor([0.6094, 0.6051], dtype=torch.float64),
 tensor([0.6094, 0.6051], dtype=torch.float64),
 tensor([0.6094, 0.6051], dtype=torch.float64),
 tensor([0.6094, 0.6051], dtype=torch.float64),
 tensor([0.6094, 0.6051], dtype=torch.float64)]

In [58]:
candidate_list

[tensor([[0.4465, 0.7681]]),
 tensor([[0.3890, 0.8014]]),
 tensor([[0.7310, 0.4068]]),
 tensor([[0.6135, 0.5706]]),
 tensor([[0.7884, 0.7705]])]

In [ ]:
beta :1.6262626262626263 noise_level :1.0
Next point: tensor([[0.6094, 0.6051]]) 
 Next Y tensor([0.4208], dtype=torch.float64)
best_x:(0.6093829870223999, 0.6050794124603271)Best_y,0.4208345751761849